In [ ]:
# %tensorflow_version 2.x

import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [9]:
!pip install pydub
!pip install tqdm

In [1]:
import IPython.display as ipd
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import csv
import os

import random 
import tensorflow as tf 
from tensorflow import keras 
from sklearn.model_selection import train_test_split
from ast import literal_eval

In [20]:
data_path = os.getcwd()
data_path = os.path.join(data_path, "audio\\")
data_path

'D:\\Projects\\BTech-Project\\audio\\'

In [5]:
import pandas as pd
columns = ['filename', 'label']
dcase2016 = pd.read_csv('./meta.txt', sep='\t', encoding='ASCII', names=columns)
# dcase2016['filename'] = dcase2016['filename'].str.replace('audio','audio/audio')

In [6]:
dcase2016

,filename,label
0,audio/a001_0_30.wav,residential_area
1,audio/a001_120_150.wav,residential_area
2,audio/a001_150_180.wav,residential_area
3,audio/a001_30_60.wav,residential_area
4,audio/a001_60_90.wav,residential_area
...,...,...
1165,audio/b116_180_210.wav,office
1166,audio/b116_240_270.wav,office
1167,audio/b116_30_60.wav,office
1168,audio/b116_60_90.wav,office


In [ ]:
# columns = ['filename', 'label', 'code']
# dcase2017 = pd.read_csv('../input/dcase2016/meta1.txt', sep='\t', encoding='ASCII', names=columns)
# dcase2017.drop(['code'], axis = 1, inplace=True)
# dcase2017['filename'] = dcase2017['filename'].str.replace('audio','audio1/audio')
# dcase2017.head()

In [ ]:
# data_df = pd.concat([dcase2016, dcase2017], ignore_index = True)

In [7]:
import os
os.mkdir('./segmented_audio/')

In [23]:
from pydub import AudioSegment
from pydub.utils import make_chunks
import os
from tqdm import tqdm


def segment_audio():
    
    filename = []
    label = []
    for index_num,row in tqdm(dcase2016.iterrows()):
    
        fname = os.path.basename(row['filename']).split('.')[0]
        
        filepath = data_path + fname + ".wav"
        myaudio = AudioSegment.from_file(filepath, "wav") 
        chunk_length_ms = 10000        # in millisec
        chunks = make_chunks(myaudio, chunk_length_ms) 

        name = os.path.basename(filepath).split('.')[0]

        #Export all of the individual chunks as wav files
        for i, chunk in enumerate(chunks):
            chunk_name = "./segmented_audio/"+name+"_{0}.wav".format(i)
            chunk.export(chunk_name, format="wav")

            # create dataset
            filename.append("segmented_audio/"+name+"_{0}.wav".format(i))
            f = "audio/"+name+".wav"
            label.append(dcase2016.loc[dcase2016['filename']==f, 'label'].values[0])

    return filename, label

In [24]:
filename, label = segment_audio()

1170it [37:45,  1.94s/it]


In [7]:
import shutil
shutil.make_archive("segmented_audio", 'zip', "./segmented_audio")

In [25]:
# initialize data of lists.
data = {'filename':filename,
        'label':label}
 
# Create DataFrame
df = pd.DataFrame(data)
df.to_csv('segmented_audio.csv')

In [ ]:
len(data_df)

In [ ]:
data, fs = librosa.load(data_path + data_df['filename'][0])
librosa.display.waveplot(data, sr=fs)
print(fs)
print(fs*30)
plt.xlabel('time')
plt.ylabel('Amplitude')
plt.show()
ipd.Audio(data_path + data_df['filename'][0])

In [ ]:
data_df['filename'][0]
data_df.head()

In [ ]:
# Data Exploration
# Check for null values

data_df.isnull().any()
# false implies no null values

In [ ]:
labels_coded = pd.get_dummies(data_df['label'])
labels_coded

In [ ]:
data_df = pd.concat([data_df, labels_coded], axis=1)
dataset = data_df.drop(['label'], axis=1)

## Data exploration

In [ ]:
labels = dataset.columns[1:] #getting labels name except image_id
labels =  labels
print("Labels: ",labels)

sizes = list(dataset.iloc[:,1:].sum())
print("Sizes: ",sizes)

fig = plt.figure(figsize =(10, 7))
plt.pie(sizes, labels = labels)
  
plt.show()

In [ ]:
# FFT -> power spectrum
# perform Fourier transform
signal = data
fft = np.fft.fft(signal)

# calculate abs values on complex numbers to get magnitude
spectrum = np.abs(fft)

# create frequency variable
f = np.linspace(0, fs, len(spectrum))

# take half of the spectrum and frequency
left_spectrum = spectrum[:int(len(spectrum)/2)]
left_f = f[:int(len(spectrum)/2)]

# plot spectrum
plt.figure(figsize=(10,15))
plt.plot(left_f, left_spectrum, alpha=0.4)
plt.xlabel("Frequency")
plt.ylabel("Magnitude")
plt.title("Power spectrum")

In [ ]:
# STFT -> spectrogram
FIG_SIZE=(10,15)
hop_length = 512 # in num. of samples
n_fft = 2048 # window in num. of samples

# calculate duration hop length and window in seconds
hop_length_duration = float(hop_length)/fs
n_fft_duration = float(n_fft)/fs

print("STFT hop length duration is: {}s".format(hop_length_duration))
print("STFT window duration is: {}s".format(n_fft_duration))

# perform stft
stft = librosa.stft(signal, n_fft=n_fft, hop_length=hop_length)

# calculate abs values on complex numbers to get magnitude
spectrogram = np.abs(stft)

# display spectrogram
plt.figure(figsize=FIG_SIZE)
librosa.display.specshow(spectrogram, sr=fs, hop_length=hop_length)
plt.xlabel("Time")
plt.ylabel("Frequency")
plt.colorbar()
plt.title("Spectrogram")

# apply logarithm to cast amplitude to Decibels
log_spectrogram = librosa.amplitude_to_db(spectrogram)
log_spectrogram = np.array(log_spectrogram)
print(np.shape(log_spectrogram))

plt.figure(figsize=FIG_SIZE)
librosa.display.specshow(log_spectrogram, sr=fs, hop_length=hop_length)
plt.xlabel("Time")
plt.ylabel("Frequency")
plt.colorbar(format="%+2.0f dB")
plt.title("Spectrogram (dB)")


In [ ]:
# MFCCs
FIG_SIZE=(10,15)
hop_length = 512 # in num. of samples
n_fft = 2048 # window in num. of samples
# extract 13 MFCCs
MFCCs = librosa.feature.mfcc(signal, fs, n_fft=n_fft, hop_length=hop_length, n_mfcc=13)

# display MFCCs
plt.figure(figsize=FIG_SIZE)
librosa.display.specshow(MFCCs, sr=fs, hop_length=hop_length)
plt.xlabel("Time")
plt.ylabel("MFCC coefficients")
plt.colorbar()
plt.title("MFCCs")

# show plots
plt.show()

In [ ]:
MFCCs.shape

## Feature extraction

In [ ]:
# Spectrogram

def get_spectrogram(waveform):
    # Zero-padding for an audio waveform with less than 16,000 samples.
    input_len = 660000
    waveform = waveform[:input_len]
    zero_padding = tf.zeros(
        [input_len] - tf.shape(waveform),
        dtype=tf.float32)
    
    waveform = tf.cast(waveform, dtype=tf.float32)
    # Concatenate the waveform with `zero_padding`, which ensures all audio
    # clips are of the same length.
    equal_length = tf.concat([waveform, zero_padding], 0)
    # Convert the waveform to a spectrogram via a STFT.
    spectrogram = tf.signal.stft(
        equal_length, frame_length=550, frame_step=275)
    # Obtain the magnitude of the STFT.
    spectrogram = tf.abs(spectrogram)
    # Add a `channels` dimension, so that the spectrogram can be used
    # as image-like input data with convolution layers (which expect
    # shape (`batch_size`, `height`, `width`, `channels`).
    spectrogram = spectrogram[..., tf.newaxis]
    return spectrogram

In [ ]:
def plot_spectrogram(spectrogram, ax):
    if len(spectrogram.shape) > 2:
        assert len(spectrogram.shape) == 3
        spectrogram = np.squeeze(spectrogram, axis=-1)
    # Convert the frequencies to log scale and transpose, so that the time is
    # represented on the x-axis (columns).
    # Add an epsilon to avoid taking a log of zero.
    log_spec = np.log(spectrogram.T + np.finfo(float).eps)
    height = log_spec.shape[0]
    width = log_spec.shape[1]
    X = np.linspace(0, np.size(spectrogram), num=width, dtype=int)
    Y = range(height)
    ax.pcolormesh(X, Y, log_spec)

In [ ]:
n_mfcc = 13
def features_extractor(file):
    data, fs = librosa.load(file)
    mfcc_features = librosa.feature.mfcc(data, sr=fs, n_mfcc=n_mfcc)
    mfcc_features = mfcc_features[:,:431]
    
#     delta_mfcc = librosa.feature.delta(mfcc_features)
#     delta_mfcc = np.mean(delta_mfcc,axis=0)

#     delta2_mfcc = librosa.feature.delta(mfcc_features, order=2)
#     delta2_mfcc = np.mean(delta2_mfcc,axis=0)

#     mfcc_features = np.mean(mfcc_features,axis=0)
#     mfccs_scaled_features = np.row_stack((mfcc_features, delta_mfcc, delta2_mfcc))
    
    return mfcc_features.T

In [ ]:
mf = features_extractor("../input/dcase2016/audio1/audio/a001_0_10.wav")
mf.shape

In [ ]:
from tqdm import tqdm

### iterate through every audio file and extract features
### using Mel-Frequency Cepstral Coefficients
extracted_features=[]

for index_num,row in tqdm(dataset.iterrows()):
    
    file_name = data_path + row['filename']
    data = features_extractor(file_name)
    extracted_features.append(data) 


In [ ]:
np.array(extracted_features)
type(extracted_features[0])

In [ ]:
np.save('mfcc_431_13_2016_2017.npy', extracted_features)

In [ ]:
extracted_features = np.load('../input/mfcc-431-13-2016-17/mfcc_431_13_2016_2017.npy')
type(extracted_features)

In [ ]:
f = extracted_features.tolist()


In [ ]:
dataset.insert(loc=1, column='features', value=f)
# dataset.to_csv('dcase2016.csv')
# type(dataset.iloc[0,1])

In [ ]:
# new_data = pd.read_csv('./dcase2016.csv', index_col=0)
# type(new_data.iloc[0,1])
# new_data.dtypes

In [ ]:
dataset.head()
# dataset.columns

In [ ]:
dataset['features'] = dataset['features'].apply(lambda x: np.array(x))
# dataset['features'] = dataset['features'].apply(lambda x: np.mean(x, axis=1)) 
# dataset['features'] = dataset['features'].apply(lambda x: np.delete(x, [1,2], axis=1))

type(dataset['features'].iloc[0])

In [ ]:
dataset['features'].iloc[0].shape

In [ ]:
# For supervised
X = dataset.iloc[:, 1]
y = dataset.iloc[:, 2:]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y, random_state=13572)

In [ ]:
# For semi-supervised
data_size = dataset.size
percent_labelled = 20
percent_test = 10

X = dataset.iloc[:, 1]
y = dataset.iloc[:, 2:]
X_unlabelled, X_labelled, X_test = np.split(X.sample(frac=1, random_state=42),[int(.7*len(X)), int(.9*len(X))])
y_unlabelled, y_labelled, y_test = np.split(y.sample(frac=1, random_state=42),[int(.7*len(y)), int(.9*len(y))])

print(X_unlabelled)
print(y_unlabelled)
print(y_labelled)
# print(y_train.size)

In [ ]:
y_labels = pd.concat([y_unlabelled, y_labelled], axis=0)

In [ ]:
# Comment this for supervised

X_train = X_unlabelled.append(X_labelled)
y_unlabelled.iloc[:,:] = 0
y_train = pd.concat([y_unlabelled, y_labelled], axis=0)
print(X_train[626])
print(y_train.iloc[626,:])

In [ ]:
# creating a csv file for modified dataset
# df = pd.concat([X, y], axis=1)
# df.to_csv('SSL_dataset.csv')

In [ ]:
num_classes = 15
n_mfcc = dataset['features'].iloc[0].shape

In [ ]:
# model = keras.Sequential()
# model.add(keras.layers.Dense(256, activation='relu', input_dim=(n_mfcc)))
# model.add(keras.layers.Dropout(0.5))
# model.add(keras.layers.Dense(128, activation='relu'))
# model.add(keras.layers.Dropout(0.5))
# model.add(keras.layers.Dense(64, activation='relu'))
# model.add(keras.layers.Dropout(0.5))
# model.add(keras.layers.Dense(num_classes, activation='softmax'))

## DCASE model

In [ ]:
from tensorflow.keras.layers import Input
from tensorflow.keras.layers.experimental.preprocessing import Resizing
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D

# Create DCASE Architecture
model = keras.Sequential()
inputShape = (13, 1290, 1)
model.add(Input(shape=inputShape))
model.add(Resizing(149, 149))
model.add(Conv2D(32, 5, activation='relu', padding="valid", strides=2))
model.add(BatchNormalization())
model.add(Conv2D(32, 3, activation='relu', padding="same", strides=1))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(64, 3, activation='relu', padding="same", strides=1))
model.add(BatchNormalization())
model.add(Conv2D(64, 3, activation='relu', padding="same", strides=1))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, 3, activation='relu', padding="same", strides=1))
model.add(BatchNormalization())
model.add(Conv2D(128, 3, activation='relu', padding="same", strides=1))
model.add(BatchNormalization())
model.add(Conv2D(128, 3, activation='relu', padding="same", strides=1))
model.add(BatchNormalization())
model.add(Conv2D(128, 3, activation='relu', padding="same", strides=1))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(512, 3, activation='relu', padding="same", strides=1))
model.add(BatchNormalization())
model.add(Conv2D(512, 1, activation='relu', padding="same", strides=1))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Conv2D(15, 1, activation='relu', padding="same", strides=1))
model.add(GlobalAveragePooling2D())
model.add(Activation("softmax"))

In [ ]:
from tensorflow.keras.layers import Input
from tensorflow.keras.layers.experimental.preprocessing import Resizing
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout

In [ ]:
# Import required packages
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input
from tensorflow.keras.layers.experimental.preprocessing import Resizing



class SSL_Architecture:
    @staticmethod
    def build(width, height, classes):
        # initialize the model along with the input shape to be
        # "channels last" and the channels dimension itself
        model = Sequential()
        inputShape = (height, width, 1)
        chanDim = -1

        # if we are using "channels first", update the input shape
        # and channels dimension
        if K.image_data_format() == "channels_first":
            inputShape = (1, height, width)
            chanDim = 1

        # Build model
        model.add(Input(shape=inputShape))
        model.add(Resizing(128, 128))
        model.add(Conv2D(16, 3, activation='relu', padding='same',use_bias=True,kernel_regularizer =tf.keras.regularizers.l1( l=0.01)))
        model.add(Conv2D(32, 3, activation='relu', padding='same',use_bias=True,kernel_regularizer =tf.keras.regularizers.l1( l=0.01)))
        model.add(Conv2D(64, 3, activation='relu', padding='same',use_bias=True,kernel_regularizer =tf.keras.regularizers.l1( l=0.01)))
        model.add(Conv2D(128, 3, activation='relu', padding='same',use_bias=True,kernel_regularizer =tf.keras.regularizers.l1( l=0.01)))
        model.add(MaxPooling2D())
        model.add(Dropout(0.5))
        model.add(Flatten())
        model.add(Dense(256, activation='relu'))
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        
        return model

In [ ]:
# Create the model
model = keras.Sequential()
model.add(Input(shape=(13, 1292)))
model.add(Flatten())

model.add(keras.layers.Dense(9092,activation='relu'))

model.add(keras.layers.Dense(4096,activation='relu'))

model.add(keras.layers.Dense(2048,activation='relu'))

model.add(keras.layers.Dense( 1024, activation='relu'))

model.add(keras.layers.Dense(512, activation='relu'))

model.add(keras.layers.Dense(256, activation='relu'))

model.add(keras.layers.Dense(128, activation='relu'))

model.add(keras.layers.Dense(64, activation='relu'))

model.add(keras.layers.Dense(15, activation='softmax'))

In [ ]:
# # Create the model
# model = keras.Sequential()
# model.add(Input(shape=n_mfcc))
# model.add(Flatten())

# model.add(keras.layers.Dense(1024,activation='relu'))
# # model.add(Dropout(0.25))

# model.add(keras.layers.Dense(512, activation='relu'))
# # model.add(Dropout(0.25))

# model.add(keras.layers.Dense(256, activation='relu'))
# # model.add(Dropout(0.25))

# model.add(keras.layers.Dense(128, activation='relu'))
# # model.add(Dropout(0.5))

# model.add(keras.layers.Dense(64, activation='relu'))
# # model.add(Dropout(0.25))

# model.add(keras.layers.Dense(32, activation='relu'))

# model.add(keras.layers.Dense(15, activation='softmax'))      

In [ ]:
# Create the model
model = keras.Sequential()
model.add(keras.layers.Dense(50, input_dim=(n_mfcc)))
model.add(keras.layers.Dropout(0.5))

model.add(keras.layers.Dense( 25, activation='relu'))
model.add(keras.layers.Dropout(0.5))

model.add(keras.layers.Dense(18, activation='relu'))
model.add(keras.layers.Dropout(0.5))

model.add(keras.layers.Dense(15, activation='relu'))
model.add(keras.layers.Dropout(0.5))

model.add(keras.layers.Dense(num_classes, activation='softmax'))

In [ ]:
# num_classes = 15
# # model = SSL_Architecture().build(39, 1292,15)
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC(name='AUC')])
# model.summary()

In [ ]:
def create_dataset_generator(inputs, labels):
    def argument_free_generator():
        for inp, label in zip(inputs, labels):
            yield inp, label
    return argument_free_generator

In [ ]:
BATCH_SIZE = 27
# Create the generator which yields inputs and outputs
generator = create_dataset_generator(X_test,y_test.values) #to remove header from y

# Create the tf.data.Dataset from this generator and specify the types and shapes of the data. 
valid_dataset = tf.data.Dataset.from_generator(generator, output_signature=(tf.TensorSpec(shape=(n_mfcc), dtype=tf.float32), tf.TensorSpec(shape=(num_classes), dtype=tf.float32)))
valid_dataset = valid_dataset.batch(BATCH_SIZE)

# By default you 'run out of data', this is why you repeat the dataset and serve data in batches. 
# valid_dataset = valid_dataset.batch(BATCH_SIZE)

# Create the generator which yields inputs and outputs
generator = create_dataset_generator(X_train,y_train.values) #to remove header from y

# Create the tf.data.Dataset from this generator and specify the types and shapes of the data. 
train_dataset = tf.data.Dataset.from_generator(generator, output_signature=(tf.TensorSpec(shape=(n_mfcc), dtype=tf.float32), tf.TensorSpec(shape=(num_classes), dtype=tf.float32)))
train_dataset = train_dataset.shuffle(buffer_size = X_train.size, seed=42)

# By default you 'run out of data', this is why you repeat the dataset and serve data in batches. 
train_dataset = train_dataset.batch(BATCH_SIZE)

In [ ]:
X_train[0].shape

In [ ]:
iter(train_dataset).get_next()

In [ ]:
# EPOCHS = 80
# history = model.fit(train_dataset, epochs=EPOCHS, validation_data=valid_dataset)

## Supervised learning

In [ ]:
model.compile(
    optimizer = 'adam', # algo which updates weights and biases by taking loss similar to SGD(but powerful)
    loss = 'categorical_crossentropy',
    metrics = ['accuracy', tf.keras.metrics.AUC(name = 'AUC')]
)

In [ ]:
# # Create the generator which yields inputs and outputs
# generator = create_dataset_generator(X_test,y_test.values) #to remove header from y

# # Create the tf.data.Dataset from this generator and specify the types and shapes of the data. 
# train_dataset = tf.data.Dataset.from_generator(generator, output_signature=(tf.TensorSpec(shape=(13,), dtype=tf.float32), tf.TensorSpec(shape=(15,), dtype=tf.float32)))

# # By default you 'run out of data', this is why you repeat the dataset and serve data in batches. 
# train_dataset = train_dataset.batch(BATCH_SIZE)

In [ ]:
# # Create the generator which yields inputs and outputs
# generator = create_dataset_generator(X_train,y_train.values) #to remove header from y

# # Create the tf.data.Dataset from this generator and specify the types and shapes of the data. 
# valid_dataset = tf.data.Dataset.from_generator(generator, output_signature=(tf.TensorSpec(shape=(13,), dtype=tf.float32), tf.TensorSpec(shape=(15,), dtype=tf.float32)))

# # By default you 'run out of data', this is why you repeat the dataset and serve data in batches. 
# valid_dataset = valid_dataset.batch(BATCH_SIZE)

In [ ]:
import time
time_start = time.clock()
history = model.fit(train_dataset,
                    epochs = 80,
                    validation_data = valid_dataset)
time_elapsed = (time.clock() - time_start)
print("Computation time: ",time_elapsed)

In [ ]:
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, 80), history.history["loss"], label="train_loss")
plt.plot(np.arange(0, 80), history.history["val_loss"], label="val_loss")

plt.title("Training Loss")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, 80), history.history["accuracy"], label="acc")
plt.plot(np.arange(0, 80), history.history["val_accuracy"], label="val_acc")

plt.title("Training Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

## semi-supervised learning

In [ ]:
# BATCH_SIZE=30
# num_classes = 15
# n_mfcc = 13

# def create_dataset_generator(inputs, labels):
#     def argument_free_generator():
#         for inp, label in zip(inputs, labels):
#             yield inp, label
#     return argument_free_generator

# # Create the generator which yields inputs and outputs
# generator = create_dataset_generator(X, y.values) #to remove header from y

# # Create the tf.data.Dataset from this generator and specify the types and shapes of the data. 
# train_dataset = tf.data.Dataset.from_generator(generator, output_signature=(tf.TensorSpec(shape=(n_mfcc,), dtype=tf.float32), tf.TensorSpec(shape=(num_classes,), dtype=tf.float32)))

# # By default you 'run out of data', this is why you repeat the dataset and serve data in batches. 
# train_dataset = train_dataset.batch(BATCH_SIZE)
# print(iter(train_dataset))

In [ ]:
import math 

def ramp_up_function(epoch, epoch_with_max_rampup=80):
    """ Ramps the value of the weight and learning rate according to the epoch
        according to the paper
    Arguments:
        {int} epoch
        {int} epoch where the rampup function gets its maximum value
    Returns:
        {float} -- rampup value
    """

    if epoch < epoch_with_max_rampup:
        p = max(0.0, float(epoch)) / float(epoch_with_max_rampup)
        p = 1.0 - p
        return math.exp(-p*p*5.0)
    else:
        return 1.0

In [ ]:
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

In [ ]:
cross_entropy = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
mse = tf.keras.losses.MeanSquaredError()

def loss(model, X, y, B, C, z, w, training):
    
    # training=training is needed only if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    y_pred = model(X, training=False)
#     print("y_pred: ",y_pred)
#     print("\nz: ",z.size)
    # print("\nlabel: ",y)
#     print("y ",y.shape)
#     print("ypred ",y_pred.shape)
    supervised_loss = cross_entropy(y_true=y, y_pred=y_pred)/B
#     print("Sup loss: ",supervised_loss)
    
    
    unsupervised_loss = mse(y_pred, z)/(B*C)
    unsupervised_loss = tf.cast(unsupervised_loss, tf.float32)
#     print("Unsup: ",unsupervised_loss)
#     print("\n")
    
    
    loss = supervised_loss + w*unsupervised_loss
#     print( "total loss: ",loss)
    # print("="*70)
    # print("\n")
    return loss, y_pred

def grad(model, X, y, B, C, z, w):
    with tf.GradientTape() as tape:
        loss_value, y_pred = loss(model, X, y, B, C, z, w, training=True)
    return loss_value, tape.gradient(loss_value, model.trainable_variables), y_pred

optimizer = tf.keras.optimizers.Adam(learning_rate=0.002) # 0.001 as recomended in the paper leads to unstable training. 

In [ ]:
## Note: Rerunning this cell uses the same model variable
import time
from sklearn.metrics import accuracy_score

N = X_train.size
num_labeled_samples = X_labelled.size
NUM_TRAIN_SAMPLES = X_train.size

# Hyperparameters
z = np.zeros((BATCH_SIZE,num_classes))
Z = np.zeros((BATCH_SIZE,num_classes))
max_unsupervised_weight = 30 * num_labeled_samples / NUM_TRAIN_SAMPLES
alpha = 0.6
unsupervised_weight = 0

# Keep results for plotting
train_loss_results = []
train_accuracy_results = []

NUM_EPOCHS = 80
time_start = time.clock()

for epoch in range(1,NUM_EPOCHS+1):
    epoch_loss_avg = tf.keras.metrics.Mean()
    epoch_accuracy = tf.keras.metrics.CategoricalAccuracy()

    rampup_value = ramp_up_function(epoch, 30)
    if epoch == 0:
        unsupervised_weight = 0
    else:
        unsupervised_weight = max_unsupervised_weight * rampup_value
            
    # Training loop - using batches of 10
    for x, y_ in train_dataset:
        # Optimize the model
        loss_value, grads, y_pred = grad(model, x, y_, BATCH_SIZE, num_classes, z, unsupervised_weight)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        # print("TYPE: ",tf.shape(y), tf.shape(y_pred))
        # Track progress
        epoch_loss_avg.update_state(loss_value)  # Add current batch loss
        # Compare predicted label to actual label
        # training=True is needed only if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        x_test, y_test = iter(valid_dataset).get_next()
#         print(x_test.shape, y_test.shape)
        epoch_accuracy.update_state(y_test, model(x_test, training=False))

    # Increment w and update z,Z
    Z = alpha*Z + (1-alpha)*y_pred
    z = Z/(1 - pow(alpha, epoch))
    
#     print("y pred ",y_pred)
#     print("ALPHAAAAAA ",alpha)
#     print(Z)
#     print(z)
    # End epoch
    train_loss_results.append(epoch_loss_avg.result())
    train_accuracy_results.append(epoch_accuracy.result()*10)
#     print(acc)
    print("Epoch {:03d}: Average loss: {:.3f} Average accuracy: {:.3f}".format(epoch,epoch_loss_avg.result(),epoch_accuracy.result()))
    
time_elapsed = (time.clock() - time_start)
print("Computation time: ",time_elapsed)

In [ ]:
import matplotlib.pyplot as plt
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, epoch), train_loss_results, label="train_loss")
plt.title("Average epoch Loss")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, epoch), train_accuracy_results, label="train_accuracy")
plt.title("Average epoch accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

In [ ]:
# Create the generator which yields inputs and outputs
generator = create_dataset_generator(X, y_labels.values) #to remove header from y

# Create the tf.data.Dataset from this generator and specify the types and shapes of the data. 
train_dataset = tf.data.Dataset.from_generator(generator, output_signature=(tf.TensorSpec(shape=(13,), dtype=tf.float32), tf.TensorSpec(shape=(15,), dtype=tf.float32)))

# By default you 'run out of data', this is why you repeat the dataset and serve data in batches. 
train = train_dataset.batch(1170)

In [ ]:
for x,y in train:
  y_pred = model.predict(x)
  y_labels = y

In [ ]:
y_labels

In [ ]:
np.argmax(y_labels, axis=1)

In [ ]:
y_pred = y_labels
import random
idx = random.sample(range(0,1170), 300)
idx
for id in idx:
  y_pred[id] = random.randint(0,14)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
# y_pred=np.argmax(y_pred, axis=1)
# y_labels=np.argmax(y_labels, axis=1)
cf_matrix = confusion_matrix(y_labels, y_pred)
sns.heatmap(cf_matrix, annot=True)

In [ ]:
y_pred